In [141]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [142]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [143]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [144]:
%%capture pwd
!pwd

In [145]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('are-we-really-witnessing-a-great-resignation', 300, 500)

# Fig 1

In [146]:
df = pd.read_excel("raw/Figure1.xlsx")
df.columns = [
    "weekx",
    "week3",
    "quit_week",
    "m_quit_week",
    "min_quit_week",
    "max_quit_week",
]

In [147]:
f = "fig1_quit_rates"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,weekx,week3,quit_week,m_quit_week,min_quit_week,max_quit_week
0,2020-01-01,5/1/20,0.140353,0.178205,0.155836,0.194676
1,2020-01-08,2,0.137055,0.148771,0.118975,0.193847
2,2020-01-15,3,0.141797,0.182289,0.153109,0.212340
3,2020-01-22,4,0.154109,0.156144,0.124798,0.194772
4,2020-01-29,2/2/20,0.127832,0.161945,0.114957,0.194068


In [148]:
ymin = 0
ymax = 0.4
glyph = "quit_week"
grey = colors["eco-gray"]
green = colors["eco-turquiose"]
base = (
    alt.Chart(f1, title="")
    .encode(
        alt.X(
            "weekx:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        )
    )
    .transform_calculate(max0="max(datum.max_" + glyph + ",datum." + glyph + ")")
    .transform_calculate(min0="min(datum.min_" + glyph + ",datum." + glyph + ")")
)

marea = base.mark_area(opacity=0.15, color=grey).encode(
    alt.Y(
        "max_" + glyph + ":Q",
        axis=alt.Axis(
            title="%",
            #             domain=False,
            gridOpacity=0.4,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=21,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".1f",
        ),
        scale=alt.Scale(domain=[ymin, ymax]),
    ),
    alt.Y2("min_" + glyph + ":Q"),
)
mline = base.mark_line(
    stroke=grey, strokeWidth=1, strokeDash=[0, 2, 2], opacity=0.8
).encode(alt.Y("m_" + glyph + ":Q", scale=alt.Scale(domain=[ymin, ymax])))
area1 = base.mark_area(opacity=0.3, color=green).encode(
    alt.Y("max0:Q", scale=alt.Scale(domain=[ymin, ymax])), alt.Y2("max_" + glyph + ":Q")
)
area2 = base.mark_area(opacity=0.3, color=green).encode(
    alt.Y("min0:Q", scale=alt.Scale(domain=[ymin, ymax])), alt.Y2("min_" + glyph + ":Q")
)
line = base.mark_line(stroke=green).encode(
    alt.Y(glyph + ":Q", scale=alt.Scale(domain=[ymin, ymax]))
)

base = alt.Chart(pd.DataFrame([{"x": "2020-01-29"}, {"x": "2020-03-18"}])).encode()
rule = base.mark_rule(
    stroke=grey, strokeWidth=1, strokeDash=[0, 1, 1], opacity=0.3
).encode(alt.X("x:T"))
base = alt.Chart(
    pd.DataFrame(
        [
            {
                "x": "2020-01-29",
                "y": 0.37,
                "text": "➡ First COVID-related death in the UK",
            },
            {"x": "2020-03-18", "y": 0.34, "text": "➡ Start of first UK lockdown"},
        ]
    )
).encode()
rlabels = base.mark_text(
    fontWeight="lighter", color=grey, align="left", baseline="bottom", fontSize=10
).encode(
    alt.X("x:T"),
    alt.Y("y:Q", scale=alt.Scale(domain=[ymin, ymax])),
    alt.Text(
        "text:N",
    ),
)
base = alt.Chart(pd.DataFrame([{"x": "2020-10-05", "y": 0.09, "text": "2020/21"}])).encode()
mlabel = base.mark_text(
    fontWeight="bold", color=green, align="left", baseline="bottom", fontSize=11
).encode(
    alt.X("x:T"),
    alt.Y("y:Q"),
    alt.Text(
        "text:N",
    ),
)
base = alt.Chart(
    pd.DataFrame([{"x": "2020-10-28", "y": 0.267, "text": "2014-2019 average"}])
).encode()
label1 = base.mark_text(
    fontWeight="lighter", color=grey, align="left", baseline="bottom", fontSize=10
).encode(alt.X("x:T"), alt.Y("y:Q"), alt.Text("text:N"))
base = alt.Chart(
    pd.DataFrame([{"x": "2020-10-28", "y": 0.25, "text": "and min-max range"}])
).encode()
label2 = base.mark_text(
    fontWeight="lighter",
    color=grey,
    align="left",
    baseline="bottom",
    fontSize=10,
    opacity=0.5,
).encode(alt.X("x:T"), alt.Y("y:Q"), alt.Text("text:N"))
layer1 = alt.layer(
    rule,
    rlabels,
    marea,
    mline,
    area1,
    area2,
    line,
    mlabel,
    label1,
    label2
).configure_view(
    stroke=None,
    height=350,
    width=500,
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [149]:
df = pd.read_excel("raw/Figure2.xls")
df.columns = [
    "weekx",
    "week3",
    "hired_week",
    "m_hired_week",
    "min_hired_week",
    "max_hired_week",
]

In [150]:
f = "fig2_hired"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,weekx,week3,hired_week,m_hired_week,min_hired_week,max_hired_week
0,2020-01-01,5/1/20,0.285889,0.450801,0.437461,0.524964
1,2020-01-08,2,0.417781,0.470451,0.395795,0.621434
2,2020-01-15,3,0.383968,0.654590,0.471119,0.711957
3,2020-01-22,4,0.545865,0.511097,0.454275,0.705582
4,2020-01-29,2/2/20,0.385378,0.613449,0.519870,0.852360


In [151]:
ymin = 0
ymax = 1.4
glyph = "hired_week"
grey = colors["eco-gray"]
green = colors["eco-turquiose"]
base = (
    alt.Chart(f2, title="")
    .encode(
        alt.X(
            "weekx:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        )
    )
    .transform_calculate(max0="max(datum.max_" + glyph + ",datum." + glyph + ")")
    .transform_calculate(min0="min(datum.min_" + glyph + ",datum." + glyph + ")")
)

marea = base.mark_area(opacity=0.15, color=grey).encode(
    alt.Y(
        "max_" + glyph + ":Q",
        axis=alt.Axis(
            title="%",
            #             domain=False,
            gridOpacity=0.4,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=21,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".1f",
        ),
        scale=alt.Scale(domain=[ymin, ymax]),
    ),
    alt.Y2("min_" + glyph + ":Q"),
)
mline = base.mark_line(
    stroke=grey, strokeWidth=1, strokeDash=[0, 2, 2], opacity=0.8
).encode(alt.Y("m_" + glyph + ":Q", scale=alt.Scale(domain=[ymin, ymax])))
area1 = base.mark_area(opacity=0.3, color=green).encode(
    alt.Y("max0:Q", scale=alt.Scale(domain=[ymin, ymax])), alt.Y2("max_" + glyph + ":Q")
)
area2 = base.mark_area(opacity=0.3, color=green).encode(
    alt.Y("min0:Q", scale=alt.Scale(domain=[ymin, ymax])), alt.Y2("min_" + glyph + ":Q")
)
line = base.mark_line(stroke=green).encode(
    alt.Y(glyph + ":Q", scale=alt.Scale(domain=[ymin, ymax]))
)

base = alt.Chart(pd.DataFrame([{"x": "2020-01-29"}, {"x": "2020-03-18"}])).encode()
rule = base.mark_rule(
    stroke=grey, strokeWidth=1, strokeDash=[0, 1, 1], opacity=0.3
).encode(alt.X("x:T"))
base = alt.Chart(
    pd.DataFrame(
        [
            {
                "x": "2020-01-29",
                "y": 1.3,
                "text": "➡ First COVID-related death in the UK",
            },
            {"x": "2020-03-18", "y": 1.22, "text": "➡ Start of first UK lockdown"},
        ]
    )
).encode()
rlabels = base.mark_text(
    fontWeight="lighter", color=grey, align="left", baseline="bottom", fontSize=10
).encode(
    alt.X("x:T"),
    alt.Y("y:Q", scale=alt.Scale(domain=[ymin, ymax])),
    alt.Text(
        "text:N",
    ),
)
base = alt.Chart(pd.DataFrame([{"x": "2020-08-15", "y": 0.29, "text": "2020/21"}])).encode()
mlabel = base.mark_text(
    fontWeight="bold", color=green, align="left", baseline="bottom", fontSize=11
).encode(
    alt.X("x:T"),
    alt.Y("y:Q"),
    alt.Text(
        "text:N",
    ),
)
base = alt.Chart(
    pd.DataFrame([{"x": "2020-11-05", "y": 1.07, "text": "2014-2019 average"}])
).encode()
label1 = base.mark_text(
    fontWeight="lighter", color=grey, align="left", baseline="bottom", fontSize=10
).encode(alt.X("x:T"), alt.Y("y:Q"), alt.Text("text:N"))
base = alt.Chart(
    pd.DataFrame([{"x": "2020-11-05", "y": 1.02, "text": "and min-max range"}])
).encode()
label2 = base.mark_text(
    fontWeight="lighter",
    color=grey,
    align="left",
    baseline="bottom",
    fontSize=10,
    opacity=0.5,
).encode(alt.X("x:T"), alt.Y("y:Q"), alt.Text("text:N"))
layer1 = alt.layer(
    rule,
    rlabels,
    marea,
    mline,
    area1,
    area2,
    line,
    mlabel,
    label1,
    label2
).configure_view(
    stroke=None,
    height=350,
    width=500,
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [152]:
df = pd.read_excel("raw/Figure3_jtj.xls")
df=df[['date','jtj']]

In [153]:
f = "fig3_jtj"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,jtj
0,2016-04-01,2.055949
1,2016-04-08,1.422209
2,2016-04-15,2.134424
3,2016-04-22,1.388688
4,2016-04-29,2.637725


In [154]:
base = (
    alt.Chart(f3)
    .encode(
        x=alt.X(
            "date:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
#                 format=".0f",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=8,
                orient="bottom",
#                 labelAngle=270,
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.region+' | '+datum.year+' | '+datum.gdp_per_head+ ' £/year/head'"
    )
)
line = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-turquiose"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.7),
            alt.GradientStop(color="white", offset=0),
        ],
        x1=1,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "jtj:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridOpacity=0.4,
            title="% with job tenure less than 3 months having been in a different job last quarter",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            format="s",
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
)
layer1 = (
    ((line).properties(height=350, width=500))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

# Fig 4

In [155]:
df = pd.read_excel("raw/Figure_4.xls")
df.columns = [
    "weekx",
    "week3",
    "looknew",
    "m_looknew",
    "min_looknew",
    "max_looknew",
]

In [156]:
f = "fig4_looknew"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,weekx,week3,looknew,m_looknew,min_looknew,max_looknew
0,2020-01-01,5/1/20,5.722040,5.780573,4.026497,7.038652
1,2020-01-08,2,5.393181,6.187643,5.740975,6.679517
2,2020-01-15,3,6.449058,6.244230,5.234304,7.303279
3,2020-01-22,4,5.689105,5.998100,5.785892,7.140300
4,2020-01-29,2/2/20,5.198729,5.996000,5.666403,7.255370


In [157]:
ymin = 2
ymax = 8
glyph = "looknew"
grey = colors["eco-gray"]
green = colors["eco-turquiose"]
base = (
    alt.Chart(f4, title="")
    .encode(
        alt.X(
            "weekx:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        )
    )
    .transform_calculate(max0="max(datum.max_" + glyph + ",datum." + glyph + ")")
    .transform_calculate(min0="min(datum.min_" + glyph + ",datum." + glyph + ")")
)

marea = base.mark_area(opacity=0.15, color=grey).encode(
    alt.Y(
        "max_" + glyph + ":Q",
        axis=alt.Axis(
            title="%",
            #             domain=False,
            gridOpacity=0.4,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=13,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0f",
        ),
        scale=alt.Scale(domain=[ymin, ymax]),
    ),
    alt.Y2("min_" + glyph + ":Q"),
)
mline = base.mark_line(
    stroke=grey, strokeWidth=1, strokeDash=[0, 2, 2], opacity=0.8
).encode(alt.Y("m_" + glyph + ":Q", scale=alt.Scale(domain=[ymin, ymax])))
area1 = base.mark_area(opacity=0.3, color=green).encode(
    alt.Y("max0:Q", scale=alt.Scale(domain=[ymin, ymax])), alt.Y2("max_" + glyph + ":Q")
)
area2 = base.mark_area(opacity=0.3, color=green).encode(
    alt.Y("min0:Q", scale=alt.Scale(domain=[ymin, ymax])), alt.Y2("min_" + glyph + ":Q")
)
line = base.mark_line(stroke=green).encode(
    alt.Y(glyph + ":Q", scale=alt.Scale(domain=[ymin, ymax]))
)

base = alt.Chart(pd.DataFrame([{"x": "2020-01-29"}, {"x": "2020-03-18"}])).encode()
rule = base.mark_rule(
    stroke=grey, strokeWidth=1, strokeDash=[0, 1, 1], opacity=0.3
).encode(alt.X("x:T"))
base = alt.Chart(
    pd.DataFrame(
        [
            {
                "x": "2020-01-29",
                "y": 2.1,
                "text": "➡ First COVID-related death in the UK",
            },
            {"x": "2020-03-18", "y": 2.4, "text": "➡ Start of first UK lockdown"},
        ]
    )
).encode()
rlabels = base.mark_text(
    fontWeight="lighter", color=grey, align="left", baseline="bottom", fontSize=10
).encode(
    alt.X("x:T"),
    alt.Y("y:Q", scale=alt.Scale(domain=[ymin, ymax])),
    alt.Text(
        "text:N",
    ),
)
base = alt.Chart(pd.DataFrame([{"x": "2020-09-11", "y": 3.4, "text": "2020/21"}])).encode()
mlabel = base.mark_text(
    fontWeight="bold", color=green, align="left", baseline="bottom", fontSize=11
).encode(
    alt.X("x:T"),
    alt.Y("y:Q"),
    alt.Text(
        "text:N",
    ),
)
base = alt.Chart(
    pd.DataFrame([{"x": "2020-11-05", "y": 7.1, "text": "2014-2019 average"}])
).encode()
label1 = base.mark_text(
    fontWeight="lighter", color=grey, align="left", baseline="bottom", fontSize=10
).encode(alt.X("x:T"), alt.Y("y:Q"), alt.Text("text:N"))
base = alt.Chart(
    pd.DataFrame([{"x": "2020-11-05", "y": 6.85, "text": "and min-max range"}])
).encode()
label2 = base.mark_text(
    fontWeight="lighter",
    color=grey,
    align="left",
    baseline="bottom",
    fontSize=10,
    opacity=0.5,
).encode(alt.X("x:T"), alt.Y("y:Q"), alt.Text("text:N"))
layer1 = alt.layer(
    rule,
    rlabels,
    marea,
    mline,
    area1,
    area2,
    line,
    mlabel,
    label1,
    label2
).configure_view(
    stroke=None,
    height=350,
    width=500,
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)